In [2]:
import sys
sys.path.append('../..')
import pandas as pd
import src.cleaning as clean
import numpy as np
import seaborn as sns
import src.analysis.utils as utils

In [3]:
sales_data = pd.read_csv('../../data/anon_data.csv')
cleaned_data = clean.clean_data(sales_data)

C:\Users\C\AppData\Local\Temp\ipykernel_11444\1800904834.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  sales_data = pd.read_csv('../../data/anon_data.csv')


In [4]:
# Assume order date is the earliest shipping date
cleaned_data['order_date'] = cleaned_data.groupby('sales_order_number')['ship_date'].transform(min)

In [5]:
#Restrict to only orders being sent not returned - being quantity > 0 
positive_quantity = cleaned_data.loc[cleaned_data['quantity']>0, :]

# Most Frequent Customers

In [6]:
# Number of order days by customer, restriction being quantity > 0 
customer_order_dates = positive_quantity.groupby('customer_id')['order_date'].unique()
customer_regularity = customer_order_dates.apply(len).sort_values(ascending=False)
top_10_regular_customers = customer_regularity.iloc[list(range(0, 10))]
top_10_regular_customers

customer_id
100002803    499
100001590    487
100007970    487
100004472    480
100001570    460
100001046    451
100000985    408
100009033    407
100008245    406
100005771    400
Name: order_date, dtype: int64

In [11]:
top_10_regular_customers.sum()

4485

In [12]:
customer_regularity[customer_regularity>customer_regularity.quantile(0.9)]

customer_id
100002803    499
100001590    487
100007970    487
100004472    480
100001570    460
            ... 
100020244     57
100013216     57
100006770     56
100014962     56
100005620     56
Name: order_date, Length: 329, dtype: int64

In [21]:
average_days_between_customer_orders = customer_order_dates.loc[top_10_regular_customers.index].apply(utils.calculate_days_between)
average_days_between_customer_orders.apply(np.mean)

customer_id
100002803    1.419679
100001590    1.454733
100007970    1.454733
100004472    1.471816
100001570    1.540305
100001046    1.568889
100000985    1.734644
100009033    1.731527
100008245    1.738272
100005771    1.761905
Name: order_date, dtype: float64

# Regularly Ordered by Regulars

In [20]:
positive_quantity_by_regulars = positive_quantity.loc[positive_quantity['customer_id'].isin(top_10_regular_customers.index), :]
positive_quantity_by_regulars.groupby('customer_id')['order_date'].unique().apply(len).sort_values(ascending=False).sum()

4485

In [15]:
# Groupby brand and item
dates_of_items_ordered = positive_quantity_by_regulars.groupby(['brand', 'product_type'])['order_date'].unique()
number_of_days_item_ordered = dates_of_items_ordered.apply(len).sort_values(ascending=False)
most_commonly_ordered_items = number_of_days_item_ordered.loc[number_of_days_item_ordered.iloc[list(range(10))].index]
most_commonly_ordered_items

brand      product_type           
Poly       Headset                    438
Dell       Monitors                   436
Lenovo     Notebook Computers         423
           Monitors                   391
Dell       Keyboard and Mouse         388
Microsoft  Office Suite Software      387
Philips    Monitors                   382
Dell       Mouse/Trackballs           378
LG         Monitors                   377
Lenovo     Professional Desktop PC    377
Name: order_date, dtype: int64

In [16]:
most_commonly_ordered_items.sum()

3977

In [9]:
days_between_items_ordered = dates_of_items_ordered[most_commonly_ordered_items.index].apply(utils.calculate_days_between).apply(np.mean).sort_values(ascending=True)
days_between_items_ordered

brand      product_type           
Poly       Headset                    1.615561
Dell       Monitors                   1.622989
Lenovo     Notebook Computers         1.677725
           Monitors                   1.812821
Microsoft  Office Suite Software      1.821244
Dell       Keyboard and Mouse         1.824289
Philips    Monitors                   1.845144
Dell       Mouse/Trackballs           1.864721
LG         Monitors                   1.867021
Lenovo     Professional Desktop PC    1.869681
Name: order_date, dtype: float64